In [22]:
from vote_system import confront_dataset, filter_pairs, round, get_website_ids, get_element_by_id
from response_parsing import parse_stable, parse_gemma, parse_qwen_coder, parse_deepseek
import json

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
codes = parse_stable("datasets/dataset_stable-code_intermediate.json", "166_com")
for code in codes : 
    print(code)


<meta name="viewport" content="width=device-width, initial-scale=1.0">
nav { flex-direction: column; } nav a { padding: 10px; }
width: 100%;

initial-scale
width
device-width
nav
column
nav a


In [4]:
code_qwen = parse_qwen_coder("datasets/dataset_qwen2-5-coder_intermediate.json", "1c_ru")

In [5]:
code_qwen

['@media (max-width: 768px) {',
 'html, body {     margin: 0;     padding: 0; }',
 'body {     font-family: Arial, sans-serif; }',
 '.container {     width: 90%;     max-width: 1200px;     margin: 0 auto; }',
 'nav ul {     list-style-type: none;     padding: 0;     display: flex;     justify-content: space-between; }',
 '@media (max-width: 576px) {     nav ul {         flex-direction: column;     } }',
 'section, footer {     padding: 20px; }',
 'footer {     background-color: #333;     color: white;     text-align: center;     position: fixed;     width: 100%;     bottom: 0; }']

In [100]:
code_gemma = parse_gemma("datasets/dataset_gemma_intermediate.json", "1c_ru")

In [101]:
code_gemma

['<',
 '**HTML:**',
 '',
 '- Use media queries with breakpoints for different devices.',
 '- Utilize flexible layouts with flexible boxes or grid systems.',
 '- Include responsive image sizes with the `srcset',
 '',
 '',
 '**CSS:**',
 '',
 '- Use relative units like `%` or `em',
 '- Employ responsive navigation menus that collapse on smaller screens.',
 '- Design for different viewport sizes and adjust layouts accordingly.',
 '- Use responsive font sizes that adapt to screen sizes.',
 '- Consider using responsive plugins like Bootstrap or Foundation.',
 '']

In [102]:
ids = get_website_ids("datasets/dataset_stable-code_intermediate.json")

In [87]:
for i in range(10):
    print(ids[i])
    cl_sc = parse_stable("datasets/dataset_stable-code_intermediate.json", ids[i])
    cl_gemma = parse_gemma("datasets/dataset_gemma_intermediate.json", ids[i])

    print(filter_pairs(confront_dataset(cl_sc, cl_gemma), threshold=0.1))

    print('=====================')

166_com
[('<meta name="viewport" content="width=device-width, initial-scale=1.0">', '@media (max-width: 600px)', 0.13333333333333333), ('<meta name="viewport" content="width=device-width, initial-scale=1.0">', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', 1.0), ('<meta name="viewport" content="width=device-width, initial-scale=1.0">', 'width: 100%', 0.15384615384615385), ('width: 100%;', '@media (max-width: 600px)', 0.3333333333333333), ('width: 100%;', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', 0.15384615384615385), ('width: 100%;', 'width: 100%', 1.0), ('initial-scale', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', 0.3076923076923077), ('width', '@media (max-width: 600px)', 0.4), ('width', '<meta name="viewport" content="width=device-width, initial-scale=1.0">', 0.16666666666666669), ('width', 'width: 100%', 0.6666666666666666), ('device-width', '@media (max-width: 600px)', 0.3333333333333333), ('d

In [8]:
def tournament_for_website(website_id, threshold=0.1):
    cl_sc = parse_stable("datasets/dataset_stable-code_intermediate.json", website_id)
    cl_gemma = parse_gemma("datasets/dataset_gemma_intermediate.json", website_id)
    cl_qwen = parse_qwen_coder("datasets/dataset_qwen2-5-coder_intermediate.json", website_id)
    cl_deep = parse_deepseek("datasets/dataset_deepseek-coder_intermediate.json", website_id)
    
    round1a_winner = round(cl_sc, cl_gemma, threshold=threshold)
    round1b_winner = round(cl_qwen, cl_deep, threshold=threshold)

    round2_winner = round(round1a_winner, round1b_winner, threshold=threshold)
    
    return round2_winner

In [10]:
tournament_for_website("166_com", threshold=0.1)

d:\Developpement\web2code-m\tournament\response_parsing.py:78: SyntaxWarning: invalid escape sequence '\s'
  responsive_lines = re.findall("(@media.*\s*)| (`*[\.\#][^!}]*[!};])| !([^!]+)!|\`{3}([^\`]*)\`{3}|<([^>]+)>|^!([^!\n]*)", responsive, re.MULTILINE)


['<meta name="viewport" content="width=device-width, initial-scale=1.0">',
 'width: 100%;',
 'initial-scale',
 'width',
 'device-width']

In [24]:
def build_dataset(path, threshold=0.1):
    ids = get_website_ids("datasets/dataset_deepseek-coder_intermediate.json")
    dataset = []
    with open("datasets/dataset_deepseek-coder_intermediate.json", 'r') as file:
        website_data = json.load(file)

        for website_id in ids:
            print(website_id)
            winners = tournament_for_website(website_id, threshold=threshold)
            if (len(winners) == 0):
                continue

            website = get_element_by_id(website_data, website_id)

            data = {
                "website_id": website_id, 
                "html_code": website["html_code"], 
                "css_code": website["css_code"], 
                "responsive_explanation": winners
            }

            dataset.append(data)

    with open(path, 'w') as file:
        json_object = json.dump(dataset, file)
        file.write(json_object)

    return dataset

In [25]:
build_dataset("datasets/dataset_merged.json", threshold=0.1)

166_com
1c_ru
1drv_com


KeyboardInterrupt: 